<a href="https://colab.research.google.com/github/itimes-digital/deep-learning-estudo/blob/main/classificacao_binaria_simples_breast_cancer_validacao_cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [2]:
previsores = pd.read_csv("https://raw.githubusercontent.com/itimes-digital/deep-learning-estudo/main/dataset/entradas_breast.csv", sep=",")
previsores.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,radius_se,texture_se,perimeter_se,area_se,smoothness_se,compactness_se,concavity_se,concave_points_se,symmetry_se,fractal_dimension_se,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1095.0000,0.9053,8589.0,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3398.0,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4585.0,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1156.0000,3445.0,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.0000,0.10430,0.1809,0.05883,0.7572,0.7813,5438.0,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,205.0000,0.4000,0.1625,0.2364,0.07678


In [3]:
previsores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0    radius_mean              569 non-null    float64
 1    texture_mean             569 non-null    float64
 2    perimeter_mean           569 non-null    float64
 3    area_mean                569 non-null    float64
 4    smoothness_mean          569 non-null    float64
 5    compactness_mean         569 non-null    float64
 6    concavity_mean           569 non-null    float64
 7   concave_points_mean       569 non-null    float64
 8    symmetry_mean            569 non-null    float64
 9    fractal_dimension_mean   569 non-null    float64
 10   radius_se                569 non-null    float64
 11   texture_se               569 non-null    float64
 12   perimeter_se             569 non-null    float64
 13   area_se                  569 non-null    float64
 14   smoothnes

In [4]:
classe = pd.read_csv("https://raw.githubusercontent.com/itimes-digital/deep-learning-estudo/main/dataset/saidas_breast.csv")
classe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       569 non-null    int64
dtypes: int64(1)
memory usage: 4.6 KB


In [5]:
classe.head()

,0
0,0
1,0
2,0
3,0
4,0


In [12]:
import keras

In [13]:
from keras.models import Sequential
from keras.layers import Dense

In [14]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

Definição do números de neurônios por camada:

(numero de entrada + numero de saídas esperada) / 2 = número de units na camada.

In [8]:
# Medida base para definir a quantidade de neurônios na camada.
# Como a classificação é binária, a resposta será 0 ou 1, portanto, uma saída.
quantSaidaEsperada = 1
units = (previsores.columns.size + quantSaidaEsperada) / 2
units = np.round(units)
units

16.0

In [15]:
def criarRede():
  # input_dim define a quantidade de neurônios para a camada de entrada.
  classificador = Sequential() # Estrutura básica para colocar as camadas sequencialmente
  classificador.add(Dense(units=16, # define a quantidade de neurônios a camada oculta 
                        activation='relu', 
                        kernel_initializer='random_uniform', 
                        input_dim=30));# Quantidade de atributos de entrada

  # Camada intermediária ou oculta
  classificador.add(Dense(units=16, # define a quantidade de neurônios a camada oculta
                        activation='relu', 
                        kernel_initializer='random_uniform'));

  # Definindo a camada de saída
  # Como é um classificador binário para 0 ou 1
  # A quantidade de neurônio é 1
  classificador.add(Dense(units=1, activation="sigmoid"))

  otimizador = keras.optimizers.Adam(lr=0.0001, decay = 0.00001, clipvalue = 0.5)
  # para classificação binária deve usar binary_crossentropy
  # para classificação com mais de duas classes deve usar categorical_crossentropy
  classificador.compile(optimizer= otimizador, 
                      loss='binary_crossentropy',
                      metrics=['binary_accuracy']);
                  
  return classificador; 


In [16]:
classificador = KerasClassifier(build_fn = criarRede,
                                epochs = 100,
                                batch_size = 10);
classificador

In [17]:
resultados = cross_val_score(estimator = classificador,
                             X = previsores, 
                             y = classe,
                             cv = 10,
                             scoring = 'accuracy')

Epoch 1/100
52/52 [==============================] - 0s 1ms/step - loss: 1.8136 - binary_accuracy: 0.3848
Epoch 2/100
52/52 [==============================] - 0s 962us/step - loss: 0.7064 - binary_accuracy: 0.4941
Epoch 3/100
52/52 [==============================] - 0s 1ms/step - loss: 0.5726 - binary_accuracy: 0.6797
Epoch 4/100
52/52 [==============================] - 0s 1ms/step - loss: 0.5368 - binary_accuracy: 0.7129
Epoch 5/100
52/52 [==============================] - 0s 1ms/step - loss: 0.5104 - binary_accuracy: 0.7012
Epoch 6/100
52/52 [==============================] - 0s 988us/step - loss: 0.4907 - binary_accuracy: 0.7207
Epoch 7/100
52/52 [==============================] - 0s 1ms/step - loss: 0.4746 - binary_accuracy: 0.7363
Epoch 8/100
52/52 [==============================] - 0s 1ms/step - loss: 0.4613 - binary_accuracy: 0.7715
Epoch 9/100
52/52 [==============================] - 0s 1ms/step - loss: 0.4507 - binary_accuracy: 0.7832
Epoch 10/100
52/52 [======================

In [20]:
resultados.mean()

0.8559210526315789

In [22]:
resultados.std()

0.051840268468662916